In [2]:
import pandas as pd
import csv

/Users/dhkim/.pyenv/versions/xai/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [9]:
num1 = pd.read_csv('data/2020-08-31_18_27_crawl_diningcode_review.csv', sep='\t').drop('Unnamed: 0', axis=1)
num2 = pd.read_csv('data/2020-08-31_18_36_crawl_diningcode_review.csv', sep='\t').drop('Unnamed: 0', axis=1)
num3 = pd.read_csv('data/2020-08-31_18_39_crawl_diningcode_review.csv', sep='\t').drop('Unnamed: 0', axis=1)
num4 = pd.read_csv('data/2020-08-31_18_41_crawl_diningcode_review.csv', sep='\t').drop('Unnamed: 0', axis=1)


In [15]:
total_dinning_review = pd.concat([num1, num2, num3, num4])

In [22]:
total_dinning_review.loc[total_dinning_review.name.map(lambda x: '나폴레옹' in x), 'name'] = '나폴레옹'

In [24]:
total_dinning_review.loc[total_dinning_review.name.map(lambda x: '경천' in x), 'name'] = '경천애인'

In [29]:
total_dinning_review.to_excel('data/dinningcode_review.xlsx', index=False, encoding='utf-8')

In [52]:
total_dinning_review.name.unique().shape

(36,)

In [44]:
like1 = pd.read_csv('data/2020-08-31_20_10_crawl_diningcode_favor_review.csv', sep='\t').drop('Unnamed: 0', axis=1)
like2 = pd.read_csv('data/2020-08-31_20_19_crawl_diningcode_favor_review.csv', sep='\t').drop('Unnamed: 0', axis=1)

In [45]:
total_dinning_like = pd.concat([like1, like2])

In [47]:
total_dinning_like.loc[total_dinning_like.name.map(lambda x: '나폴레옹' in x), 'name'] = '나폴레옹'

In [55]:
total_dinning_like.loc[total_dinning_like.name.map(lambda x: '경천' in x), 'name'] = '경천애인'

In [67]:
total_dinning_like.loc[2259] = ''
total_dinning_like.loc[2259, 'name'] = '패트릭스와플'
total_dinning_like.loc[2259, 'person'] = 'onis (1590곳)'
total_dinning_like.loc[2259, 'like_date'] = '2019년 7월 21일'

In [68]:
total_dinning_like.to_excel('data/dinningcode_review_like.xlsx', index=False, encoding='utf-8')

In [69]:
total_dinning_like.name.unique().shape

(39,)

## 좋아요는 있는데 리뷰는 없는 곳 : 5곳

In [87]:
set(total_dinning_like.name.unique()).difference(set(total_dinning_review.name.unique()))

{'구이구이', '그레이에스프레소', '배꼽집', '신원', '하쯔호'}

## 리뷰는 있는데 좋아요는 없는 곳 : 3곳

In [88]:
set(total_dinning_review.name.unique()).difference(set(total_dinning_like.name.unique()))

{'나폴레옹', '카페진정성'}

In [89]:
place_list = pd.read_csv('../data/pilot_lists2.csv', sep=',').name
place_list = place_list.map(lambda x:'경천애인' if '경천' in x else x)
place_list = place_list.map(lambda x:'나폴레옹' if '나폴레옹' in x else x)
place_list = place_list.values

## 리뷰도 없고 좋아요도 없는 곳 : 3곳

In [91]:
crawl_total = set(total_dinning_like.name.unique()).union(set(total_dinning_review.name.unique()))

In [92]:
set(place_list).difference(crawl_total)

{'대오옛집', '브로드웨이', '스시혜정'}

In [4]:
dinning = pd.read_excel('data/dinningcode_review_like.xlsx')

In [6]:
dinning_review = pd.read_excel('data/dinningcode_review.xlsx')

In [52]:
def func(each_group):
    new_dict = {}
#     each_group.reset_index(drop=True)
    new_dict['review'] = str({i:review for i, review in enumerate(each_group.review)})
    new_dict['date'] = str({i:date for i, date in enumerate(each_group.date)})
    new_dict['score'] = str({i:score for i, score in enumerate(each_group.score)})
    return pd.DataFrame(new_dict, index=[each_group.name], columns=['review', 'date', 'score'])
    print(new_dict['reveiw'])
#     each_group.review

In [53]:
dinning_review.name.unique().shape

(36,)

In [73]:
dinning_review_reshape = dinning_review.groupby('name').apply(lambda x:func(x)).reset_index().drop('level_1', axis=1)

In [71]:
dinning_review_reshape.loc[:, 'review_num'] = dinning_review_reshape.review.map(lambda x:len(x.split(',')))
dinning_review_reshape.loc[:, 'date_num'] = dinning_review_reshape.date.map(lambda x:len(x.split(',')))
dinning_review_reshape.loc[:, 'score_num'] = dinning_review_reshape.score.map(lambda x:len(x.split(',')))

In [75]:
dinning_review_reshape.to_excel('dinning_review_reshape.xlsx', index=False)

In [76]:
pd.read_excel('dinning_review_reshape.xlsx')

,name,review,date,score
0,가양칼국수버섯매운탕,{0: '여의도 직장인이라면 대부분 알고 있는 맛집. 재료도 메뉴도 단순하기 때문에...,"{0: '2019년 3월 27일', 1: '7월 14일', 2: '2019년 6월 ...","{0: 5, 1: 3, 2: 4, 3: 4, 4: 5, 5: 5, 6: 5, 7: ..."
1,갓포아키,{0: '정갈하게 일식 먹고 싶을 때 추천합니다. 한우스키야키 우동전골 추천해요!'},{0: '2019년 8월 29일'},{0: 5}
2,경천애인,{0: '한우집으로 매우 유명하고 맛있는집 입니다. 맛의 기준이 개인적으로 다르겠지...,"{0: '2019년 8월 16일', 1: '2019년 8월 18일', 2: '7월 ...","{0: 5, 1: 4, 2: 5}"
3,그리너리샐러드,{0: '여의도 쪽에서는 가장 맛있는 샐러드집이라고 생각함! 야채 양이 정말 많아서...,"{0: '2019년 8월 29일', 1: '2019년 12월 19일', 2: '20...","{0: 4, 1: 5, 2: 3}"
4,그리츠,"{0: '이른 송년모임\n후배들이 고른 그리츠\n주중석식 55,000원이다\n\n음...",{0: '2019년 11월 16일'},{0: 3}
5,나폴레옹,{0: '서울 3대 빵집으로 유명한 곳이라 기대하고 갔어요. 유명한 사라다빵도 넉넉...,{0: '6월 8일'},{0: 5}
6,뉴오리진,"{0: ""한줄 평\n'브랜드이미지에 걸맞는 건강한 식탁'\n\n\n뉴오리진은 유한양...","{0: '2019년 9월 4일', 1: '7월 8일', 2: '2019년 6월 5일'}","{0: 5, 1: 4, 2: 5}"
7,더스테이크하우스,{0: '상하이 지점을 가보고 여의도점을 방문했다는 것을 참고바랍니다. 중국 상하이...,"{0: '2018년 6월 11일', 1: '2017년 4월 22일'}","{0: 3, 1: 4}"
8,라무진,{0: '남이 구워주는 맛있는 고기가 먹고싶은데 소 돼지가 안끌릴때 가면 딱 좋을만...,"{0: '2019년 8월 30일', 1: '2018년 4월 27일', 2: '201...","{0: 5, 1: 4, 2: 4, 3: 5}"
9,바스버거,"{0: ""한줄 평\n'본격 burger & bar'\n\n\n바스버거는 펀드매니저 ...","{0: '2019년 8월 16일', 1: '6월 16일', 2: '7월 29일', ...","{0: 4, 1: 3, 2: 3, 3: 4, 4: 4, 5: 4, 6: 4, 7: ..."


In [ ]:
def like_agg(each_group):
    

In [77]:
dinning.groupby('name').apply(lambda x:like_agg(x))

,name,person,like_date
0,스시아라타,조정환 (347곳),2020년 4월 11일
1,스시아라타,김재환 (3500곳),2020년 3월 7일
2,스시아라타,몽돌이 (22곳),2020년 2월 13일
3,스시아라타,고영 (11곳),2020년 1월 27일
4,스시아라타,TiberiusChoe (215곳),2019년 12월 1일
...,...,...,...
2255,슈치쿠,Young Wook Song (1곳),2016년 8월 31일
2256,슈치쿠,김수현여친 (314곳),2016년 2월 5일
2257,슈치쿠,luvlatte (34곳),2015년 11월 11일
2258,슈치쿠,sykimmmm (27곳),2015년 10월 29일
